In [2]:
# look at tools/set_up_magics.ipynb
yandex_metrica_allowed = True ; get_ipython().run_cell('# one_liner_str\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \'// setup cpp code highlighting\\nIPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\')\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE\nimport random\nimport sys\nimport os\nimport re\nimport signal\nimport shutil\nimport shlex\nimport glob\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                if line.startswith(run_prefix):\n                    cmds.append(line[len(run_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                run_prefix = "%# "\n                if line.startswith(run_prefix):\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                raise Exception("Unknown %%save_file subcommand: \'%s\'" % line)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        display(Markdown("Run: `%s`" % cmd))\n        get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    assert not fname\n    save_file("makefile", cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    try:\n        expr, comment = line.split(" #")\n        display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n    except:\n        display(Markdown("{} = {}".format(line, eval(line))))\n        \ndef show_file(file, clear_at_begin=True, return_html_string=False):\n    if clear_at_begin:\n        get_ipython().system("truncate --size 0 " + file)\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                elem.innerText = xmlhttp.responseText;\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (errors___OBJ__ < 10 && !entrance___OBJ__) {\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n        \n        <font color="white"> <tt>\n        <p id="__OBJ__" style="font-size: 16px; border:3px #333333 solid; background: #333333; border-radius: 10px; padding: 10px;  "></p>\n        </tt> </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__ -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n    \nBASH_POPEN_TMP_DIR = "./bash_popen_tmp"\n    \ndef bash_popen_terminate_all():\n    for p in globals().get("bash_popen_list", []):\n        print("Terminate pid=" + str(p.pid), file=sys.stderr)\n        p.terminate()\n    globals()["bash_popen_list"] = []\n    if os.path.exists(BASH_POPEN_TMP_DIR):\n        shutil.rmtree(BASH_POPEN_TMP_DIR)\n\nbash_popen_terminate_all()  \n\ndef bash_popen(cmd):\n    if not os.path.exists(BASH_POPEN_TMP_DIR):\n        os.mkdir(BASH_POPEN_TMP_DIR)\n    h = os.path.join(BASH_POPEN_TMP_DIR, str(random.randint(0, 1e18)))\n    stdout_file = h + ".out.html"\n    stderr_file = h + ".err.html"\n    run_log_file = h + ".fin.html"\n    \n    stdout = open(stdout_file, "wb")\n    stdout = open(stderr_file, "wb")\n    \n    html = """\n    <table width="100%">\n    <colgroup>\n       <col span="1" style="width: 70px;">\n       <col span="1">\n    </colgroup>    \n    <tbody>\n      <tr> <td><b>STDOUT</b></td> <td> {stdout} </td> </tr>\n      <tr> <td><b>STDERR</b></td> <td> {stderr} </td> </tr>\n      <tr> <td><b>RUN LOG</b></td> <td> {run_log} </td> </tr>\n    </tbody>\n    </table>\n    """.format(\n        stdout=show_file(stdout_file, return_html_string=True),\n        stderr=show_file(stderr_file, return_html_string=True),\n        run_log=show_file(run_log_file, return_html_string=True),\n    )\n    \n    cmd = """\n        bash -c {cmd} &\n        pid=$!\n        echo "Process started! pid=${{pid}}" > {run_log_file}\n        wait ${{pid}}\n        echo "Process finished! exit_code=$?" >> {run_log_file}\n    """.format(cmd=shlex.quote(cmd), run_log_file=run_log_file)\n    # print(cmd)\n    display(HTML(html))\n    \n    p = Popen(["bash", "-c", cmd], stdin=PIPE, stdout=stdout, stderr=stdout)\n    \n    bash_popen_list.append(p)\n    return p\n\n\n@register_line_magic\ndef bash_async(line):\n    bash_popen(line)\n    \n    \ndef show_log_file(file, return_html_string=False):\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function halt__OBJ__(elem, color)\n        {\n            elem.setAttribute("style", "font-size: 14px; background: " + color + "; padding: 10px; border: 3px; border-radius: 5px; color: white; ");                    \n        }\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            if (entrance___OBJ__ < 0) {\n                entrance___OBJ__ = 0;\n            }\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                if (elem.innerHTML != xmlhttp.responseText) {\n                                    elem.innerHTML = xmlhttp.responseText;\n                                }\n                                if (elem.innerHTML.includes("Process finished.")) {\n                                    halt__OBJ__(elem, "#333333");\n                                } else {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                }\n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (!entrance___OBJ__) {\n                                if (errors___OBJ__ < 6) {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                } else {\n                                    halt__OBJ__(elem, "#994444");\n                                }\n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n\n        <p id="__OBJ__" style="font-size: 14px; background: #000000; padding: 10px; border: 3px; border-radius: 5px; color: white; ">\n        </p>\n        \n        </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__.md -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n\n    \nclass TInteractiveLauncher:\n    tmp_path = "./interactive_launcher_tmp"\n    def __init__(self, cmd):\n        try:\n            os.mkdir(TInteractiveLauncher.tmp_path)\n        except:\n            pass\n        name = str(random.randint(0, 1e18))\n        self.inq_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".inq")\n        self.log_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".log")\n        \n        os.mkfifo(self.inq_path)\n        open(self.log_path, \'w\').close()\n        open(self.log_path + ".md", \'w\').close()\n\n        self.pid = os.fork()\n        if self.pid == -1:\n            print("Error")\n        if self.pid == 0:\n            exe_cands = glob.glob("../tools/launcher.py") + glob.glob("../../tools/launcher.py")\n            assert(len(exe_cands) == 1)\n            assert(os.execvp("python3", ["python3", exe_cands[0], "-l", self.log_path, "-i", self.inq_path, "-c", cmd]) == 0)\n        self.inq_f = open(self.inq_path, "w")\n        interactive_launcher_opened_set.add(self.pid)\n        show_log_file(self.log_path)\n\n    def write(self, s):\n        s = s.encode()\n        assert len(s) == os.write(self.inq_f.fileno(), s)\n        \n    def get_pid(self):\n        n = 100\n        for i in range(n):\n            try:\n                return int(re.findall(r"PID = (\\d+)", open(self.log_path).readline())[0])\n            except:\n                if i + 1 == n:\n                    raise\n                time.sleep(0.1)\n        \n    def input_queue_path(self):\n        return self.inq_path\n        \n    def close(self):\n        self.inq_f.close()\n        os.waitpid(self.pid, 0)\n        os.remove(self.inq_path)\n        # os.remove(self.log_path)\n        self.inq_path = None\n        self.log_path = None \n        interactive_launcher_opened_set.remove(self.pid)\n        self.pid = None\n        \n    @staticmethod\n    def terminate_all():\n        if "interactive_launcher_opened_set" not in globals():\n            globals()["interactive_launcher_opened_set"] = set()\n        global interactive_launcher_opened_set\n        for pid in interactive_launcher_opened_set:\n            print("Terminate pid=" + str(pid), file=sys.stderr)\n            os.kill(pid, signal.SIGKILL)\n            os.waitpid(pid, 0)\n        interactive_launcher_opened_set = set()\n        if os.path.exists(TInteractiveLauncher.tmp_path):\n            shutil.rmtree(TInteractiveLauncher.tmp_path)\n    \nTInteractiveLauncher.terminate_all()\n   \nyandex_metrica_allowed = bool(globals().get("yandex_metrica_allowed", False))\nif yandex_metrica_allowed:\n    display(HTML(\'\'\'<!-- YANDEX_METRICA_BEGIN -->\n    <script type="text/javascript" >\n       (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};\n       m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})\n       (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");\n\n       ym(59260609, "init", {\n            clickmap:true,\n            trackLinks:true,\n            accurateTrackBounce:true\n       });\n    </script>\n    <noscript><div><img src="https://mc.yandex.ru/watch/59260609" style="position:absolute; left:-9999px;" alt="" /></div></noscript>\n    <!-- YANDEX_METRICA_END -->\'\'\'))\n\ndef make_oneliner():\n    html_text = \'("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "")\'\n    html_text += \' + "<""!-- MAGICS_SETUP_PRINTING_END -->"\'\n    return \'\'.join([\n        \'# look at tools/set_up_magics.ipynb\\n\',\n        \'yandex_metrica_allowed = True ; get_ipython().run_cell(%s);\' % repr(one_liner_str),\n        \'display(HTML(%s))\' % html_text,\n        \' #\'\'MAGICS_SETUP_END\'\n    ])\n       \n\n');display(HTML(("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "") + "<""!-- MAGICS_SETUP_PRINTING_END -->")) #MAGICS_SETUP_END

<IPython.core.display.Javascript object>

# Инструменты разработки

### [Видеозапись семинара (пока с 2020 года, TODO)](https://www.youtube.com/watch?v=E8a0m6HG2x8&list=PLjzMm8llUm4AmU6i_hPU0NobgA4VsBowc&index=2)


[Ридинг Яковлева про компиляцию, python.ctypes, gdb](https://github.com/victor-yacovlev/mipt-diht-caos/blob/master/practice/linux_basics/devtools.md)


Сегодня в программе:
* <a href="#compile" style="color:#856024"> Компиляция и ее этапы </a>
  * <a href="#simple_compile" style="color:#856024"> Просто скомпилировать! </a>
  * <a href="#preprocess" style="color:#856024"> Прeпроцессинг </a>
  * <a href="#compilation" style="color:#856024"> Компиляция </a>
  * <a href="#assembling" style="color:#856024"> Acceмблирование </a>
  * <a href="#linking" style="color:#856024"> Компоновка </a>

* <a href="#elf" style="color:#856024"> Динамические библиотеки, объектные и исполняемые файлы </a>

* <a href="#debug" style="color:#856024"> Отладка и инструментирование </a>
  * <a href="#gdb" style="color:#856024"> GDB </a>
  * <a href="#sanitizers" style="color:#856024"> Sanitizers and valgrind </a>
    * <a href="#asan_segv" style="color:#856024"> ASAN и проезды по памяти </a>
    * <a href="#asan_leak" style="color:#856024"> ASAN: Обнаружение утечек памяти с помощью address-санитайзера </a>
  * <a href="#strace" style="color:#856024"> STRACE: Отладка системных вызовов с помощью strace </a>

* <a href="#run" style="color:#856024"> Запуск и завершение программы </a>

* <a href="#macro" style="color:#856024"> Дополнение: макросы в C/C++ </a>


## <a name="compile"></a> Компиляция и ее этапы

[Процесс компиляции программ на C++ / Хабр](https://habr.com/ru/post/478124/)

[Этапы компиляции / cppreference](https://en.cppreference.com/w/c/language/translation_phases) - тут выделены другие этапы, подробнее описано все про препроцессор.

Этапы компиляции:
1. Препроцессинг: разворачиваем инклюды и макросы
2. Компиляция: превращаем код на С/С++ в ассемблерный код
3. Ассемблирование: ассемблерный код -> машинный
4. Компоновка: сборка исполняемого файла из объектных файлов и статических библиотек

### <a name="simple_compile"></a> Просто скомпилировать!

Скомпилировать исходный код в исполняемый файл несложно: просто передаем все исходные файлы компилятору (файлы могут быть с разных этапов компиляции: исходные, после препроцессора, ассемблерный код (.S), объектные файлы (.o), ...)

In [105]:
%%cpp hello_world.c
%run gcc hello_world.c -o hello_world_c.exe
%run ./hello_world_c.exe

#include <stdio.h>

int main() {
    printf("Hello world!\n");
    return 0;
}

Run: `gcc hello_world.c -o hello_world_c.exe`

Run: `./hello_world_c.exe`

Hello world!


In [4]:
%%cpp hello_world.cpp
%run g++ hello_world.cpp -o hello_world_cpp.exe
%run ./hello_world_cpp.exe

#include <iostream>

int main() {
    std::cout << "Hello world!\n";
    return 0;
}


Run: `g++ hello_world.cpp -o hello_world_cpp.exe`

Run: `./hello_world_cpp.exe`

Hello world!


### <a name="preprocess"></a> Препроцессинг

In [5]:
%%cpp preprocessing_max.h
int f(int a, int b);

In [6]:
%%cpp preprocessing_max.c
%run gcc -E preprocessing_max.c -o preprocessing_max_E.c
%run cat preprocessing_max_E.c
#include "preprocessing_max.h"
#define max(a, b) ((a) > (b) ? (a) : (b))

int f(int a, int b) { // it's comment
    return max(a, b);
}

Run: `gcc -E preprocessing_max.c -o preprocessing_max_E.c`

Run: `cat preprocessing_max_E.c`

# 1 "preprocessing_max.c"
# 1 "<built-in>"
# 1 "<command-line>"
# 31 "<command-line>"
# 1 "/usr/include/stdc-predef.h" 1 3 4
# 32 "<command-line>" 2
# 1 "preprocessing_max.c"



# 1 "preprocessing_max.h" 1

int f(int a, int b);
# 5 "preprocessing_max.c" 2


int f(int a, int b) {
    return ((a) > (b) ? (a) : (b));
}


Результат препроцессинга является корректным кодом, его можно использовать так же, как и обычный код. Просто теперь в нем раскрыты все инструкции препроцессора

In [7]:
%%cpp preprocessing_max_main.c
%run gcc preprocessing_max_main.c preprocessing_max_E.c -o preprocessing_max.exe
%run ./preprocessing_max.exe

#include "preprocessing_max.h"

#include <stdio.h>

int main() {
    printf("max(5, 7) = %d\n", f(5, 7));
    return 0;
}

Run: `gcc preprocessing_max_main.c preprocessing_max_E.c -o preprocessing_max.exe`

Run: `./preprocessing_max.exe`

max(5, 7) = 7


### <a name="compilation"></a> Компиляция

Превратим исходный код в ассемблерный.

In [8]:
# здесь необязательно брать результат работы препроцессора
# -Os -fno-asynchronous-unwind-tables помогает получить более короткий выхлоп
# -fverbose-asm позволит получить более длинный, но более удобный для чтения
!gcc -S preprocessing_max_E.c -o preprocessing_max_E.S -Os -Wl,--gc-sections -fno-asynchronous-unwind-tables -fcf-protection=branch -mmanual-endbr 
!cat preprocessing_max_E.S 

	.file	"preprocessing_max_E.c"
	.text
	.globl	f
	.type	f, @function
f:
	cmpl	%edi, %esi
	movl	%edi, %eax
	cmovge	%esi, %eax
	ret
	.size	f, .-f
	.ident	"GCC: (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	 1f - 0f
	.long	 4f - 1f
	.long	 5
0:
	.string	 "GNU"
1:
	.align 8
	.long	 0xc0000002
	.long	 3f - 2f
2:
	.long	 0x1
3:
	.align 8
4:


Способ получить ассемблерный код функции. Правда тут не просто проводится компиляция, здесь создается машинный код, а потом gdb его обратно дизассемблирует.

In [9]:
!gcc -c preprocessing_max_E.c -o preprocessing_max.o -Os -fno-asynchronous-unwind-tables -fcf-protection=branch -mmanual-endbr 
!gdb preprocessing_max.o -batch -ex="disass f"

Dump of assembler code for function f:
   0x0000000000000000 <+0>:	cmp    %edi,%esi
   0x0000000000000002 <+2>:	mov    %edi,%eax
   0x0000000000000004 <+4>:	cmovge %esi,%eax
   0x0000000000000007 <+7>:	retq   
End of assembler dump.


Еще один способ

In [58]:
!gcc -g -c preprocessing_max.c -Os -fno-asynchronous-unwind-tables -fcf-protection=branch -mmanual-endbr 
!objdump -d -S preprocessing_max.o


preprocessing_max.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <f>:
// %run cat preprocessing_max_E.c
#include "preprocessing_max.h"
#define max(a, b) ((a) > (b) ? (a) : (b))

int f(int a, int b) { // it's comment
    return max(a, b);
   0:	39 fe                	cmp    %edi,%esi
   2:	89 f8                	mov    %edi,%eax
   4:	0f 4d c6             	cmovge %esi,%eax
}
   7:	c3                   	retq   


### <a name="assembling"></a> Ассемблирование

Ничего интересного, ассемблер и так слишком приближен к машинному коду. 

In [11]:
!gcc -c preprocessing_max_E.S -o preprocessing_max.o

### <a name="linking"></a> Компоновка / Линковка

Финальная сборка одного исполняемого файла

Она производится утилитой `ld`, но проще ее не запоминать и пользоваться gcc, который сам ее вызовет.

(А как именно вызовет можно узнать, добавив опции `-v -Wl,-v`, конструкция `-Wl,` позволяет указать опциию, которую gcc прокинет линкеру)

In [12]:
!gcc preprocessing_max.o preprocessing_max_main.c -o preprocessing_max_main.exe
!./preprocessing_max_main.exe

max(5, 7) = 7


## <a name="elf"></a> Динамические библиотеки, объектные и исполняемые файлы


[ELF / Wiki](https://en.wikipedia.org/wiki/Executable_and_Linkable_Format) - Executable and Linkable Format

In [13]:
%%cpp lib.c
%run gcc -c lib.c -o lib.o #// компилируем в объектный файл
%run ar rcs lib.a lib.o #// делаем из объектного файла статическую библиотеку (что по сути архив объектных файлов)
%run gcc -shared -fPIC lib.o -o lib.so #// делаем из объектного файла динамическую библиотеку

int sum(int a, int b) {
    return a + b;
}

float sum_f(float a, float b) {
    return a + b;
}

Run: `gcc -c lib.c -o lib.o #// компилируем в объектный файл`

Run: `ar rcs lib.a lib.o #// делаем из объектного файла статическую библиотеку (что по сути архив объектных файлов)`

Run: `gcc -shared -fPIC lib.o -o lib.so #// делаем из объектного файла динамическую библиотеку`

In [55]:
!echo lib.o:  ; hexdump -C lib.o | head -n 2  # обратите внимание на 0x7f E L F - магическое начало файла

lib.o:
00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  01 00 3e 00 01 00 00 00  00 00 00 00 00 00 00 00  |..>.............|


In [54]:
!echo lib.a:    ; hexdump -C lib.a  | head -n 2 # заметим, что это не ELF но тоже имеет магическое начало
!echo libb.so:  ; hexdump -C lib.so | head -n 2 # а это тоже ELF

lib.a:
00000000  21 3c 61 72 63 68 3e 0a  2f 20 20 20 20 20 20 20  |!<arch>./       |
00000010  20 20 20 20 20 20 20 20  30 20 20 20 20 20 20 20  |        0       |
libb.so:
00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  03 00 3e 00 01 00 00 00  40 10 00 00 00 00 00 00  |..>.....@.......|


In [57]:
!echo preprocessing_max.exe:  ; hexdump -C preprocessing_max.exe | head -n 2 # а это тоже ELF

preprocessing_max.exe:
00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  03 00 3e 00 01 00 00 00  60 10 00 00 00 00 00 00  |..>.....`.......|


In [17]:
!objdump -t lib.o | grep sum  # symbols in shared library

0000000000000000 g     F .text	0000000000000018 sum
0000000000000018 g     F .text	000000000000001e sum_f


In [18]:
!objdump -t lib.so | grep sum  # symbols in shared library

0000000000001111 g     F .text	000000000000001e sum_f
00000000000010f9 g     F .text	0000000000000018 sum


In [19]:
%%cpp main.c
%run gcc main.c lib.a -o main.exe
%run ./main.exe

#include <stdio.h>

int sum(int a, int b);

int main() {
    printf("%d", sum(1, 2));
    return 0;
}

Run: `gcc main.c lib.a -o main.exe`

Run: `./main.exe`

3

// Кандидат на выкидывание

Подгрузим динамическую библиотеку из Python

In [20]:
import ctypes

lib = ctypes.CDLL("./lib.so")
%p lib.sum(3, 4)
%p lib.sum_f(3, 4)

lib.sum_f.restype = ctypes.c_float
%p lib.sum_f(3, 4) # with set return type

lib.sum_f.argtypes = [ctypes.c_float, ctypes.c_float]
%p lib.sum_f(3, 4) # with set return and arguments types

lib.sum(3, 4) = 7

lib.sum_f(3, 4) = 0

`lib.sum_f(3, 4) = 1.328390586784154e-38`  # with set return type

`lib.sum_f(3, 4) = 7.0`  # with set return and arguments types

## <a name="debug"></a> Отладка и инструментирование

### <a name="gdb"></a> GDB

Полезные команды с моей практики:
* ```gdb -p $(pidof <your_process_name>) -ex 'set height 0' -ex 'thread apply all bt' -ex 'detach' -ex 'quit' > bt.txt``` - прям основное - снять бектрейсы всех потоков.
* `set scheduler-locking on/off` - никогда не надо забывать, если хотите вызывать функции многопоточной программы из gdb. Блокирует/включает работу других тредов (кроме того, на котором вы остановились в gdb). 


In [65]:
%%cpp segfault.cpp

#include<stdio.h>

int access(int* a, int i) { 
    return a[i]; 
}

int main() {
    int a[2] = {41, 42}, i = 100501;
    printf("a[%d] = %d\n", i, access(a, i)); // проезд по памяти
}

Бывает просто полезно запустить программу под gdb

In [66]:
# компилируем с отладочной информацией и запускаем под gdb
!gcc -g segfault.cpp -o segfault.exe
!gdb -ex=r -batch --args ./segfault.exe


Program received signal SIGSEGV, Segmentation fault.
0x000055555555518c in access (a=0x7fffffffdce0, i=100501) at segfault.cpp:6
6	    return a[i]; 


Если запустили не под GDB, но коркнулось - не беда)

In [67]:
!sudo sysctl kernel.core_pattern="./core"              # Говорим складывать корки рядом в файлик core
!ulimit -c unlimited ; ./segfault.exe                  # Устанавливаем системный лимит и запускаем. Оно коркается.
!gdb -ex "bt" -ex "p i" -batch ./segfault.exe ./core   # Смотрим на корку с помощью GDB

kernel.core_pattern = ./core
Segmentation fault (core dumped)
[New LWP 200623]
Core was generated by `./segfault.exe'.
Program terminated with signal SIGSEGV, Segmentation fault.
#0  0x000055f5eed0418c in access (a=0x7ffd1734f550, i=100501) at segfault.cpp:6
6	    return a[i]; 
#0  0x000055f5eed0418c in access (a=0x7ffd1734f550, i=100501) at segfault.cpp:6
#1  0x000055f5eed041d1 in main () at segfault.cpp:11
$1 = 100501


In [68]:
!gdb -ex "frame 1" -ex "p a" -batch ./segfault.exe ./core 

[New LWP 200623]
Core was generated by `./segfault.exe'.
Program terminated with signal SIGSEGV, Segmentation fault.
#0  0x000055f5eed0418c in access (a=0x7ffd1734f550, i=100501) at segfault.cpp:6
6	    return a[i]; 
#1  0x000055f5eed041d1 in main () at segfault.cpp:11
11	    printf("a[%d] = %d\n", i, access(a, i)); // проезд по памяти
$1 = {41, 42}


Можно вмешаться в ход программы. Например, уровень логирования так можно подрутить экстренно

In [69]:
!gdb -ex "b segfault.cpp:6" -ex "r" -ex "set var i = 0" -ex "c" -batch --args ./segfault.exe

Breakpoint 1 at 0x1178: file segfault.cpp, line 6.

Breakpoint 1, access (a=0x7fffffffdce0, i=100501) at segfault.cpp:6
6	    return a[i]; 
a[100501] = 41
[Inferior 1 (process 200633) exited normally]


### <a name="sanitizers"></a> Sanitizers

#### <a name="asan_segv"></a> ASAN и проезды по памяти

`ASAN_OPTIONS=verbosity=10 ./segfault.exe` - регулировка уровня многословности asan.

In [70]:
%%cpp segfault_access.cpp

#include<stdio.h>

int a[] = {41, 42};

int get_element(int i) {
    return a[i]; // проезд по памяти при некорректных i
}

In [71]:
%%cpp segfault_2.cpp

#include<stdio.h>

int get_element(int i);

int main() {
    printf("%d\n", get_element(100500));// проезд по памяти
}

In [72]:
# компилируем и запускаем как обычно
!gcc segfault_2.cpp segfault_access.cpp -o segfault.exe
!./segfault.exe

Segmentation fault


In [73]:
# компилируем с санитайзером и запускаем как обычно (семинарист рекомендует)
!gcc -fsanitize=address segfault_2.cpp segfault_access.cpp -o segfault.exe
!./segfault.exe

AddressSanitizer:DEADLYSIGNAL
==200656==ERROR: AddressSanitizer: SEGV on unknown address 0x55d3044042b0 (pc 0x55d30439f2c2 bp 0x7fffafcf7fc0 sp 0x7fffafcf7fb0 T0)
==200656==The signal is caused by a READ memory access.
    #0 0x55d30439f2c1 in get_element(int) (/home/pechatnov/vbox/caos/sem01-instruments-compilation-libc/segfault.exe+0x12c1)
    #1 0x55d30439f1fa in main (/home/pechatnov/vbox/caos/sem01-instruments-compilation-libc/segfault.exe+0x11fa)
    #2 0x7fb93e5dc0b2 in __libc_start_main (/lib/x86_64-linux-gnu/libc.so.6+0x270b2)
    #3 0x55d30439f12d in _start (/home/pechatnov/vbox/caos/sem01-instruments-compilation-libc/segfault.exe+0x112d)

AddressSanitizer can not provide additional info.
SUMMARY: AddressSanitizer: SEGV (/home/pechatnov/vbox/caos/sem01-instruments-compilation-libc/segfault.exe+0x12c1) in get_element(int)
==200656==ABORTING


Как работает ASAN? Генерирует код с проверками.

In [74]:
!gcc -c segfault_access.cpp -Os -fno-asynchronous-unwind-tables -fcf-protection=branch -mmanual-endbr 
!gdb segfault_access.o -batch -ex="disass get_element"

Dump of assembler code for function _Z11get_elementi:
   0x0000000000000000 <+0>:	movslq %edi,%rdi
   0x0000000000000003 <+3>:	lea    0x0(%rip),%rax        # 0xa <_Z11get_elementi+10>
   0x000000000000000a <+10>:	mov    (%rax,%rdi,4),%eax
   0x000000000000000d <+13>:	retq   
End of assembler dump.


In [75]:
!gcc --sanitize=address -c segfault_access.cpp -Os -fno-asynchronous-unwind-tables -fcf-protection=branch -mmanual-endbr 
!gdb segfault_access.o -batch -ex="disass get_element"

Dump of assembler code for function _Z11get_elementi:
   0x0000000000000000 <+0>:	movslq %edi,%rdi
   0x0000000000000003 <+3>:	lea    0x0(%rip),%rax        # 0xa <_Z11get_elementi+10>
   0x000000000000000a <+10>:	lea    (%rax,%rdi,4),%r8
   0x000000000000000e <+14>:	mov    %r8,%rdx
   0x0000000000000011 <+17>:	shr    $0x3,%rdx
   0x0000000000000015 <+21>:	mov    0x7fff8000(%rdx),%cl
   0x000000000000001b <+27>:	mov    %r8,%rdx
   0x000000000000001e <+30>:	and    $0x7,%edx
   0x0000000000000021 <+33>:	add    $0x3,%edx
   0x0000000000000024 <+36>:	cmp    %cl,%dl
   0x0000000000000026 <+38>:	jl     0x35 <_Z11get_elementi+53>
   0x0000000000000028 <+40>:	test   %cl,%cl
   0x000000000000002a <+42>:	je     0x35 <_Z11get_elementi+53>
   0x000000000000002c <+44>:	push   %rax
   0x000000000000002d <+45>:	mov    %r8,%rdi
   0x0000000000000030 <+48>:	callq  0x35 <_Z11get_elementi+53>
   0x0000000000000035 <+53>:	mov    (%rax,%rdi,4),%eax
   0x0000000000000038 <+56>:	retq   
End of assembler dump.

#### <a name="asan_leak"></a> ASAN: Обнаружение утечек памяти с помощью address-санитайзера

In [79]:
# компилируем с санитайзером и запускаем как обычно
!gcc -fsanitize=address memory_leak.cpp -o memory_leak.exe
!./memory_leak.exe


==200702==ERROR: LeakSanitizer: detected memory leaks

Direct leak of 16 byte(s) in 1 object(s) allocated from:
    #0 0x7feb149ddbc8 in malloc (/lib/x86_64-linux-gnu/libasan.so.5+0x10dbc8)
    #1 0x555a3f7f019a in main (/home/pechatnov/vbox/caos/sem01-instruments-compilation-libc/memory_leak.exe+0x119a)
    #2 0x7feb147050b2 in __libc_start_main (/lib/x86_64-linux-gnu/libc.so.6+0x270b2)

SUMMARY: AddressSanitizer: 16 byte(s) leaked in 1 allocation(s).


**Всегда используйте ASAN при тестировании ваших программ.** За исключением, пожалуй, очень тяжелых тестов и бенчмарков.

### <a name="strace"></a> STRACE: Отладка системных вызовов с помощью strace

In [82]:
%%cpp printing.cpp

#include<stdio.h>

int main() {
    printf("Hello, world!");
}

In [83]:
# компилируем как обычно и запускаем с strace
!gcc printing.cpp -o printing.exe
!strace ./printing.exe > out.txt 2> err.txt
!echo "Trace:"            ; cat err.txt | sed 's/^/    /'
!echo "Program output:"   ; cat out.txt | sed 's/^/    /'

Trace:
    execve("./printing.exe", ["./printing.exe"], 0x7ffdf75cd180 /* 68 vars */) = 0
    brk(NULL)                               = 0x5590f87f5000
    arch_prctl(0x3001 /* ARCH_??? */, 0x7ffe42af3e90) = -1 EINVAL (Invalid argument)
    access("/etc/ld.so.preload", R_OK)      = -1 ENOENT (No such file or directory)
    openat(AT_FDCWD, "/etc/ld.so.cache", O_RDONLY|O_CLOEXEC) = 3
    fstat(3, {st_mode=S_IFREG|0644, st_size=105883, ...}) = 0
    mmap(NULL, 105883, PROT_READ, MAP_PRIVATE, 3, 0) = 0x7fb809296000
    close(3)                                = 0
    openat(AT_FDCWD, "/lib/x86_64-linux-gnu/libc.so.6", O_RDONLY|O_CLOEXEC) = 3
    read(3, "\177ELF\2\1\1\3\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\0\360q\2\0\0\0\0\0"..., 832) = 832
    pread64(3, "\6\0\0\0\4\0\0\0@\0\0\0\0\0\0\0@\0\0\0\0\0\0\0@\0\0\0\0\0\0\0"..., 784, 64) = 784
    pread64(3, "\4\0\0\0\20\0\0\0\5\0\0\0GNU\0\2\0\0\300\4\0\0\0\3\0\0\0\0\0\0\0", 32, 848) = 32
    pread64(3, "\4\0\0\0\24\0\0\0\3\0\0\0GNU\0cBR\340\305\370\2609W

## <a name="run"></a> Запуск и завершение программы

[Интересная статья про запуск и завершение программы](http://dbp-consulting.com/tutorials/debugging/linuxProgramStartup.html)

![](http://dbp-consulting.com/tutorials/debugging/images/callgraph.png)
    
    
Пример из статьи.

In [91]:
%%cpp article_example.c
%run gcc article_example.c -o article_example.exe
%run ./article_example.exe

#include <stdio.h>
#include <stdlib.h>

void preinit(int argc, char **argv, char **envp) {
    printf("%s\n", __FUNCTION__);
}

void init(int argc, char **argv, char **envp) {
    printf("%s\n", __FUNCTION__);
}

void fini() {
    printf("%s\n", __FUNCTION__);
}

__attribute__((section(".init_array"))) typeof(init) *__init = init;
__attribute__((section(".preinit_array"))) typeof(preinit) *__preinit = preinit;
__attribute__((section(".fini_array"))) typeof(fini) *__fini = fini;

void  __attribute__ ((constructor)) constructor() {
    printf("%s\n", __FUNCTION__);
}

void __attribute__ ((destructor)) destructor() {
    printf("%s\n", __FUNCTION__);
}

void my_atexit() {
    printf("%s\n", __FUNCTION__);
}

void my_atexit2() {
    printf("%s\n", __FUNCTION__);
}

int main() {
    printf("%s 1\n", __FUNCTION__);
    atexit(my_atexit);
    atexit(my_atexit2);
    printf("%s 2\n", __FUNCTION__);
}

Run: `gcc article_example.c -o article_example.exe`

Run: `./article_example.exe`

preinit
init
constructor
main 1
main 2
my_atexit2
my_atexit
destructor
fini


А вы думали программы без `main` не бывает?

`sudo apt-get install libc6-dev-amd64` - надо установить, чтобы `sys/syscall.h` нашелся. 

In [92]:
%%cpp no_main_func.c
%run gcc -std=gnu11 -m32 -masm=intel -nostdlib -Os -s no_main_func.c -o no_main_func.exe 
%run ./no_main_func.exe

#include <sys/syscall.h>

    
// Универсальная функция для совершения системных вызовов
int syscall(int code, ...);
__asm__(R"(
syscall:
    push ebx
    push ebp
    push esi
    push edi
    mov eax, DWORD PTR [esp + 20] 
    mov ebx, DWORD PTR [esp + 24] 
    mov ecx, DWORD PTR [esp + 28] 
    mov edx, DWORD PTR [esp + 32]
    mov esi, DWORD PTR [esp + 36]
    mov edi, DWORD PTR [esp + 40]
    int 0x80
    pop edi
    pop esi
    pop ebp
    pop ebx
    ret
)");

// запись строки в файловый дескриптор
int print_s(int fd, const char* s) {
    int len = 0;
    while (s[len]) ++len;
    return syscall(SYS_write, fd, s, len);
}

void _start() {
    print_s(1, "Hello world from 'syscall'!\n");
    syscall(SYS_exit, 0);
}

Run: `gcc -std=gnu11 -m32 -masm=intel -nostdlib -Os -s no_main_func.c -o no_main_func.exe`

Run: `./no_main_func.exe`

Hello world from 'syscall'!


In [93]:
%%cpp main_func.c
%run gcc -std=gnu11 -Os -s main_func.c -o main_func.exe 
%run ./main_func.exe

#include <stdio.h>

int main() {
    printf("Hello world from 'syscall'!\n");
    return 0;
}

Run: `gcc -std=gnu11 -Os -s main_func.c -o main_func.exe`

Run: `./main_func.exe`

Hello world from 'syscall'!


In [94]:
!stat ./no_main_func.exe ; echo -n "\n"
!stat ./main_func.exe

  File: ./no_main_func.exe
  Size: 13076     	Blocks: 32         IO Block: 4096   regular file
Device: 805h/2053d	Inode: 4723931     Links: 1
Access: (0775/-rwxrwxr-x)  Uid: ( 1000/pechatnov)   Gid: ( 1000/pechatnov)
Access: 2022-09-07 21:42:59.123895996 +0300
Modify: 2022-09-07 21:42:58.999891249 +0300
Change: 2022-09-07 21:42:58.999891249 +0300
 Birth: -

  File: ./main_func.exe
  Size: 14472     	Blocks: 32         IO Block: 4096   regular file
Device: 805h/2053d	Inode: 4723932     Links: 1
Access: (0775/-rwxrwxr-x)  Uid: ( 1000/pechatnov)   Gid: ( 1000/pechatnov)
Access: 2022-09-07 21:43:00.047931384 +0300
Modify: 2022-09-07 21:42:59.919926482 +0300
Change: 2022-09-07 21:42:59.919926482 +0300
 Birth: -


In [95]:
!echo no_main_func.exe: && ldd ./no_main_func.exe
!echo main_func.exe: && ldd ./main_func.exe

no_main_func.exe:
	statically linked
main_func.exe:
	linux-vdso.so.1 (0x00007fffa4dea000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007f6a7c893000)
	/lib64/ld-linux-x86-64.so.2 (0x00007f6a7caa6000)


In [96]:
!perf stat ./no_main_func.exe   2>&1 | grep -E -v '<not supported>|^$' ; echo -n "\n"
!perf stat ./main_func.exe      2>&1 | grep -E -v '<not supported>|^$'

Hello world from 'syscall'!
 Performance counter stats for './no_main_func.exe':
              0,20 msec task-clock                #    0,334 CPUs utilized          
                 0      context-switches          #    0,000 K/sec                  
                 0      cpu-migrations            #    0,000 K/sec                  
                16      page-faults               #    0,081 M/sec                  
       0,000590183 seconds time elapsed
       0,000556000 seconds user
       0,000000000 seconds sys

Hello world from 'syscall'!
 Performance counter stats for './main_func.exe':
              0,85 msec task-clock                #    0,518 CPUs utilized          
                 0      context-switches          #    0,000 K/sec                  
                 0      cpu-migrations            #    0,000 K/sec                  
                53      page-faults               #    0,062 M/sec                  
       0,001649449 seconds time elapsed
       0,00145800

In [97]:
# stace на статически скомпилированню программу
!strace ./main_func.exe > out.txt 2> err.txt
!echo main_func.exe:
!echo "    Trace:"            ; cat err.txt | sed 's/^/        /' | cut -c -80
!echo "    Program output:"   ; cat out.txt | sed 's/^/        /'
!strace ./no_main_func.exe > out.txt 2> err.txt
!echo no_main_func.exe:
!echo "    Trace:"            ; cat err.txt | sed 's/^/        /' | cut -c -80
!echo "    Program output:"   ; cat out.txt | sed 's/^/        /'

main_func.exe:
    Trace:
        execve("./main_func.exe", ["./main_func.exe"], 0x7ffe6b1f7fa0 /* 68 vars
        brk(NULL)                               = 0x55f61f0b4000
        arch_prctl(0x3001 /* ARCH_??? */, 0x7fff1a22d370) = -1 EINVAL (Invalid a
        access("/etc/ld.so.preload", R_OK)      = -1 ENOENT (No such file or dir
        openat(AT_FDCWD, "/etc/ld.so.cache", O_RDONLY|O_CLOEXEC) = 3
        fstat(3, {st_mode=S_IFREG|0644, st_size=105883, ...}) = 0
        mmap(NULL, 105883, PROT_READ, MAP_PRIVATE, 3, 0) = 0x7f44547e9000
        close(3)                                = 0
        openat(AT_FDCWD, "/lib/x86_64-linux-gnu/libc.so.6", O_RDONLY|O_CLOEXEC) 
        read(3, "\177ELF\2\1\1\3\0\0\0\0\0\0\0\0\3\0>\0\1\0\0\0\360q\2\0\0\0\0\0
        pread64(3, "\6\0\0\0\4\0\0\0@\0\0\0\0\0\0\0@\0\0\0\0\0\0\0@\0\0\0\0\0\0\
        pread64(3, "\4\0\0\0\20\0\0\0\5\0\0\0GNU\0\2\0\0\300\4\0\0\0\3\0\0\0\0\0
        pread64(3, "\4\0\0\0\24\0\0\0\3\0\0\0GNU\0cBR\340\305\370\2609W\242\345)


## <a name="macro"></a> Дополнение: макросы в C/C++ </a>

[Статья про макросы / opennet](https://www.opennet.ru/docs/RUS/cpp/cpp-5.html)

* 

<details> <summary> Почему использование макросов стоит минимизировать </summary>
  <pre> <code> 
Тимур Демченко
расскажите, пожалуйста, почему объявление констант в c++ через define считается плохим кодстайлом
Yuri Pechatnov
Макросы в принципе считаются плохим кодстайлом, но они очень полезны бывают
Но не в случае #define CONST 5, так как это хорошо заменяется на constexpr int CONST = 5;
Mikhail Tsion
Because all macros (which are what #defines define) are in a single namespace and they take effect everywhere. Variables, including const-qualified variables, can be encapsulated in classes and namespaces.
Macros are used in C because in C, a const-qualified variable is not actually a constant, it is just a variable that cannot be modified. A const-qualified variable cannot appear in a constant expression, so it can't be used as an array size, for example.
In C++, a const-qualified object that is initialized with a constant expression (like const int x = 5 * 2;) is a constant and can be used in a constant expression, so you can and should use them.
Yuri Pechatnov
Ну и они вне неймспейсов, да
Если использовать много библиотек, то можно наткнуться на клеш дефайнов, или получить еще что-то удивительное в плохом смысле
  </code> </pre>
</details>


* Макросы это именно макросы, они ничего не знают про синтаксис С/С++

In [98]:
%%cpp macro_example_0.c
%run gcc -E macro_example_0.c -o macro_example_0_E.c
%run cat macro_example_0_E.c

#define people students and students
#define goodbye(var) Good bye var! 

Hello people!
#undef people
Hello people!
goodbye(bad grades)

Run: `gcc -E macro_example_0.c -o macro_example_0_E.c`

Run: `cat macro_example_0_E.c`

# 1 "macro_example_0.c"
# 1 "<built-in>"
# 1 "<command-line>"
# 31 "<command-line>"
# 1 "/usr/include/stdc-predef.h" 1 3 4
# 32 "<command-line>" 2
# 1 "macro_example_0.c"







Hello students and students!

Hello people!
Good bye bad grades!


* Переменные в макросах это просто куски исходного текста
* При передаче аргументов в макрос стоит помнить, что значение имеют только запятые `,` и скобки `()`

In [99]:
%%cpp macro_example_0_2.c
%run gcc -E macro_example_0_2.c -o macro_example_0_2_E.c
%run cat macro_example_0_2_E.c

#define macro(type, var, value) type var = value;

// #define protect(...) __VA_ARGS__

macro(protect(std::pair<int, int>), a, protect({1, 2, 3}))

Run: `gcc -E macro_example_0_2.c -o macro_example_0_2_E.c`

Run: `cat macro_example_0_2_E.c`

# 1 "macro_example_0_2.c"
# 1 "<built-in>"
# 1 "<command-line>"
# 31 "<command-line>"
# 1 "/usr/include/stdc-predef.h" 1 3 4
# 32 "<command-line>" 2
# 1 "macro_example_0_2.c"
# 9 "macro_example_0_2.c"
protect(std::pair<int, int>) a = protect({1, 2, 3});


Больше примеров

In [100]:
%%cpp macro_example.c
%run gcc macro_example.c -o macro_example.exe
%run ./macro_example.exe
%run gcc -DDEBUG macro_example.c -o macro_example.exe
%run ./macro_example.exe

#include <stdio.h>

#if !defined(DEBUG)
//#ifndef DEBUG
    #define DEBUG 0
#endif


#define CONST_A 123

#define mult(a, b) ((a) * (b))

#define mult_bad(a, b) (a * b)

// Склеивание имен
#define add_prefix_aba_(w) aba_##w

int main() {
    printf("START\n");
    #if DEBUG
        const char* file_name = "001.txt";
        printf("Read from '%s'. DEBUG define is enabled!\n", file_name);
        freopen(file_name, "rt", stdin);
    #endif

    printf("CONST_A %d\n", CONST_A);
    printf("mult(4, 6) = %d\n", mult(2 + 2, 3 + 3));
    printf("mult_bad(4, 6) = %d\n", mult_bad(2 + 2, 3 + 3));

    int aba_x = 42;
    int x = 420;
    printf("aba_x ? x = %d\n", add_prefix_aba_(x)); // aba_x
    
    printf("DEBUG = %d\n", DEBUG);

    return 0;
}

Run: `gcc macro_example.c -o macro_example.exe`

Run: `./macro_example.exe`

START
CONST_A 123
mult(4, 6) = 24
mult_bad(4, 6) = 11
aba_x ? x = 42
DEBUG = 0


Run: `gcc -DDEBUG macro_example.c -o macro_example.exe`

Run: `./macro_example.exe`

START
Read from '001.txt'. DEBUG define is enabled!
CONST_A 123
mult(4, 6) = 24
mult_bad(4, 6) = 11
aba_x ? x = 42
DEBUG = 1


И полезных примеров:

In [101]:
%%cpp macro_example_2.c
%run cat macro_example_2.c | grep -v "// %" > macro_example_2_filtered.c
%run gcc -std=c99 -ansi macro_example_2_filtered.c -o macro_example_2.exe
%run ./macro_example_2.exe
%run gcc -std=gnu99 macro_example_2.c -o macro_example_2.exe
%run ./macro_example_2.exe

#include <stdio.h>
#include <string.h>
#include <assert.h>

/* #VAR_NAME разворачивается в строковый литерал "VAR_NAME" */
#define print_int(i) printf(#i " = %d\n", (i));

/* Полезный макрос для вывода в поток ошибок */
#define eprintf(...) fprintf(stderr, __VA_ARGS__)

#define logprintf_impl(fmt, line, ...) eprintf(__FILE__ ":" #line " " fmt, __VA_ARGS__)
#define logprintf_impl_2(line, fmt, ...) logprintf_impl(fmt "%s", line, __VA_ARGS__)
#define logprintf(...) logprintf_impl_2(__LINE__, __VA_ARGS__, "")

#define SWAP(a, b) { __typeof__(a) c = (a); (a) = (b); (b) = (c); }
#define SWAP2(a, b) { char c[sizeof(a)]; memcpy(c, &a, sizeof(a)); \
                      memcpy(&a, &b, sizeof(a)); memcpy(&b, c, sizeof(a)); if (0) { a = b; b = a; } }

/* Способ сделать макрос с переменным числом аргументов
 * И это единственный способ "перегрузить функцию в С" */
#define sum_2(a, b, c) ((a) + (b))
#define sum_3(a, b, c) ((a) + (b) + (c))

#define sum_impl(a, b, c, sum_func, ...) sum_func(a, b, c)

#define sum(...) sum_impl(__VA_ARGS__, sum_3, sum_2)


int main() {
    /* assert(3 > 4); */
    print_int(9 * 9 + 1);

    eprintf("It is in stderr: %d\n", 431);

    int x = 1, y = 2;
    eprintf("(x, y) = (%d, %d)\n", x, y);
    SWAP(x, y);
    eprintf("(x, y) = (%d, %d)\n", x, y);
    SWAP2(x, y);
    eprintf("(x, y) = (%d, %d)\n", x, y);

    print_int(sum(1, 1));
    print_int(sum(1, 1, 1));
    
    eprintf("%s %s %d\n", __FILE__, __FUNCTION__, __LINE__);
    
    logprintf("Before exit %s\n", "");
    return 0;
}

Run: `cat macro_example_2.c | grep -v "// %" > macro_example_2_filtered.c`

Run: `gcc -std=c99 -ansi macro_example_2_filtered.c -o macro_example_2.exe`

Run: `./macro_example_2.exe`

9 * 9 + 1 = 82
It is in stderr: 431
(x, y) = (1, 2)
(x, y) = (2, 1)
(x, y) = (1, 2)
sum(1, 1) = 2
sum(1, 1, 1) = 3
macro_example_2_filtered.c main 46
macro_example_2_filtered.c:48 Before exit 


Run: `gcc -std=gnu99 macro_example_2.c -o macro_example_2.exe`

Run: `./macro_example_2.exe`

9 * 9 + 1 = 82
It is in stderr: 431
(x, y) = (1, 2)
(x, y) = (2, 1)
(x, y) = (1, 2)
sum(1, 1) = 2
sum(1, 1, 1) = 3
macro_example_2.c main 52
macro_example_2.c:54 Before exit 


In [102]:
!gcc -E macro_example_2.c -o out &&  cat out

# 1 "macro_example_2.c"
# 1 "<built-in>"
# 1 "<command-line>"
# 31 "<command-line>"
# 1 "/usr/include/stdc-predef.h" 1 3 4
# 32 "<command-line>" 2
# 1 "macro_example_2.c"







# 1 "/usr/include/stdio.h" 1 3 4
# 27 "/usr/include/stdio.h" 3 4
# 1 "/usr/include/x86_64-linux-gnu/bits/libc-header-start.h" 1 3 4
# 33 "/usr/include/x86_64-linux-gnu/bits/libc-header-start.h" 3 4
# 1 "/usr/include/features.h" 1 3 4
# 461 "/usr/include/features.h" 3 4
# 1 "/usr/include/x86_64-linux-gnu/sys/cdefs.h" 1 3 4
# 452 "/usr/include/x86_64-linux-gnu/sys/cdefs.h" 3 4
# 1 "/usr/include/x86_64-linux-gnu/bits/wordsize.h" 1 3 4
# 453 "/usr/include/x86_64-linux-gnu/sys/cdefs.h" 2 3 4
# 1 "/usr/include/x86_64-linux-gnu/bits/long-double.h" 1 3 4
# 454 "/usr/include/x86_64-linux-gnu/sys/cdefs.h" 2 3 4
# 462 "/usr/include/features.h" 2 3 4
# 485 "/usr/include/features.h" 3 4
# 1 "/usr/include/x86_64-linux-gnu/gnu/stubs.h" 1 3 4
# 10 "/usr/include/x86_64-linux-gnu/gnu/stubs.h" 3 4
# 1 "/usr/include/x86_64-linux-g

Можно упороться и сделать себе подобие деструкторов локальных переменных в Си:

In [103]:
%%cpp macro_local_vars.c
%run gcc -fsanitize=address macro_local_vars.c -o macro_local_vars.exe
%run echo -n "Hello123" > a.txt
%run ./macro_local_vars.exe

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>

struct defer_record {
    struct defer_record* previous;
    void (*func) (void*);
    void* arg;
};

#define _EXECUTE_DEFERRED(to) do { \
    while (last_defer_record != to) { \
        last_defer_record->func(last_defer_record->arg); \
        last_defer_record = last_defer_record->previous; \
    } \
} while (0)

// Интересная особенность, но здесь нужна глубина раскрытия 2, чтобы __LINE__ правильно подставился
#define _DEFER_NAME_2(line) defer_record_ ## line
#define _DEFER_NAME(line) _DEFER_NAME_2(line)

// Добавляем элемент в односвязанный список отложенных функций
#define DEFER(func, arg) \
    struct defer_record _DEFER_NAME(__LINE__) = {last_defer_record, (void (*)(void*))func, (void*)arg}; \
    last_defer_record = &_DEFER_NAME(__LINE__);

// DFB = Defer Friendly Block
// Запоминаем начала блока, в котором может использоватсья DEFER (но не во вложенных блоках!)
#define DFB_BEGIN \
    struct defer_record* first_defer_record = last_defer_record; \
    { \
        struct defer_record* last_defer_record = first_defer_record; 

// Конец блока (выполнение отложенных функций)
#define DFB_END \
        _EXECUTE_DEFERRED(first_defer_record); \
    } 

// Запоминаем начала блока функции
#define DFB_FUNCTION_BEGIN \
    struct defer_record* last_defer_record = NULL; \
    DFB_BEGIN 

// Запоминаем начала блока следующего после for, while, do
#define DFB_BREAKABLE_BEGIN \
    struct defer_record* first_breakable_defer_record = last_defer_record; \
    DFB_BEGIN

// DF = Defer Friendly
#define DF_RETURN(value) { \
    _EXECUTE_DEFERRED(NULL); \
    return value; \
}

#define DF_BREAK { \
    _EXECUTE_DEFERRED(first_breakable_defer_record); \
    break; \
} 


void func(int i) { DFB_FUNCTION_BEGIN
    void* data = malloc(145); DEFER(free, data);
    void* data2 = malloc(14); DEFER(free, data2);
    if (i % 10 == 0) {
        DF_RETURN();
    }
    if (i % 4 == 0) {
        while (1) { DFB_BREAKABLE_BEGIN
            void* data = malloc(145); DEFER(free, data);
            if (++i > 99) {
                DF_BREAK;
            }
        DFB_END }
        
        DF_RETURN();
    }
    
DFB_END } 

int main() { DFB_FUNCTION_BEGIN 
    void* data = malloc(145); DEFER(free, data);
    
    { DFB_BEGIN   
        void* data = malloc(145); DEFER(free, data);
    DFB_END }
            
    { DFB_BEGIN   
        int fd = open("a.txt", O_RDONLY);
        if (fd < 1) {
            fprintf(stderr, "Can't open file\n");
            DF_RETURN(-1);
        }
        DEFER(close, (size_t)fd);
        char buff[10];
        int len = read(fd, buff, sizeof(buff) - 1);
        buff[len] = '\0';
        printf("Read string '%s'\n", buff);
    DFB_END }
        
    
    for (int i = 0; i < 100; ++i) { DFB_BREAKABLE_BEGIN    
        void* data = malloc(145); DEFER(free, data);
        if (i % 10 == 0) { DFB_BEGIN
            DF_BREAK;
        DFB_END }
    DFB_END }
            
    DF_RETURN(0);
DFB_END }

Run: `gcc -fsanitize=address macro_local_vars.c -o macro_local_vars.exe`

Run: `echo -n "Hello123" > a.txt`

Run: `./macro_local_vars.exe`

Read string 'Hello123'
